In [1]:
data_var = '2024-01-26'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12279,2024-01-26,Eua Nba,00:00,Golden State Warriors,Sacramento Kings,2.20,1.78,242.5,1.85,2.09,1.5,2.09,1.85,jq0BT9ds,0.454545,0.561798,0.540541,0.478469,0.016343,0.8,0.2,NaN,NaN,307.092,203.337696,0.662139,244.238,107.734320,0.441104,215.344,248.350,195.64,310.05,0.0,0.0,0.0,0.0,0.149239,0.086145,0.086145,-0.65,-0.130,-9.230769,0.634790,0.6,-0.034790,-2.32,-0.464,-1.681034,0.544187,0.5,-0.044187
12280,2024-01-26,Eua Nba,00:30,Los Angeles Lakers,Chicago Bulls,1.53,2.77,227.5,1.97,1.97,-6.5,2.01,1.73,UZbFSTBm,0.653595,0.361011,0.507614,0.507614,0.014606,0.4,0.6,NaN,NaN,253.390,143.917007,0.567966,227.416,71.955971,0.316407,197.842,270.916,159.46,346.91,0.0,0.0,0.0,0.0,0.407820,0.000000,0.105877,0.01,0.002,265.000000,0.628005,0.6,-0.028005,-1.70,-0.340,-5.205882,0.344879,0.4,0.055121
12281,2024-01-26,Eua Nba,21:00,Atlanta Hawks,Dallas Mavericks,1.95,1.85,239.5,1.83,1.83,-2.5,1.90,1.90,b93JRmRg,0.512821,0.540541,0.546448,0.546448,0.053361,0.0,0.6,NaN,NaN,294.302,105.622762,0.358892,248.342,21.645353,0.087159,159.166,345.672,183.35,245.30,0.0,0.0,0.0,0.0,0.037216,0.000000,0.000000,-1.97,-0.394,-2.411168,0.585787,0.4,-0.185787,-0.03,-0.006,-141.666667,0.466977,0.5,0.033023
12282,2024-01-26,Eua Nba,21:00,Charlotte Hornets,Houston Rockets,2.78,1.45,224.5,1.80,2.02,4.5,2.03,1.79,x02NQ7t0,0.359712,0.689655,0.555556,0.495050,0.049367,0.2,0.6,NaN,NaN,613.936,546.268420,0.889781,481.916,392.097506,0.813622,327.842,459.754,411.18,385.25,1.0,0.0,0.0,0.0,0.444658,0.081447,0.088851,-3.44,-0.688,-2.587209,0.371013,0.3,-0.071013,-0.53,-0.106,-4.245283,0.378255,0.4,0.021745
12283,2024-01-26,Eua Nba,21:00,Indiana Pacers,Phoenix Suns,2.83,1.51,240.5,1.80,2.02,4.5,2.04,1.91,KOiSPRe6,0.353357,0.662252,0.555556,0.495050,0.015609,0.4,0.6,NaN,NaN,306.142,117.123006,0.382577,191.392,49.466182,0.258455,247.130,246.796,353.76,276.75,1.0,0.0,0.0,0.0,0.430129,0.081447,0.046544,0.55,0.110,16.636364,0.589264,0.6,0.010736,2.24,0.448,1.138393,0.546629,0.6,0.053371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12401,2024-01-26,Eslovênia 1. Skl Feminino,17:00,Ilirija F,Triglav F,2.53,1.48,131.5,1.85,1.85,5.5,1.85,1.85,fiTBBk5D,0.395257,0.675676,0.540541,0.540541,0.070933,0.0,0.0,NaN,NaN,242.314,344.373308,1.421186,269.730,421.688331,1.563372,0.000,0.000,79.31,74.74,0.0,0.0,0.0,0.0,0.370305,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12402,2024-01-26,Finlândia I Divisão A,14:00,Raiders Basket,Pyrinto Tampere 2,1.37,2.91,159.5,1.85,1.85,-10.5,1.95,1.72,ClsGbNcC,0.729927,0.343643,0.540541,0.540541,0.073570,0.0,0.0,NaN,NaN,171.578,72.252965,0.421109,247.190,138.479293,0.560214,0.000,0.000,105.09,207.36,0.0,0.0,0.0,0.0,0.508853,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12403,2024-01-26,Geórgia Copa Da Geórgia,11:00,Orbi,Kavkasia,1.59,2.24,166.5,1.87,1.87,-4.5,2.09,1.66,lxPVYzib,0.628931,0.446429,0.534759,0.534759,0.075359,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.240010,0.000000,0.162163,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12404,2024-01-26,Noruega Blno,15:00,Kongsberg Miners,Fyllingen,2.90,1.38,170.5,1.85,1.83,5.5,1.99,1.71,l0NWkrGF,0.344828,0.724638,0.540541,0.

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Sacramento Kings,242.5,1.85,1.0000,Over
1,21:00,Eua Nba,Indiana Pacers,Phoenix Suns,240.5,1.80,0.9946,Over
2,22:00,Eua Nba,Milwaukee Bucks,Cleveland Cavaliers,237.5,1.81,0.9912,Over
3,16:30,Europa Euroliga,Milano,Barcelona,156.5,1.91,0.8390,Over
